In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
import cubic_spline_planner


NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5  # horizon length

# mpc parameters
R = np.diag([0.01, 0.01])  # input cost matrix
Rd = np.diag([0.01, 1.0])  # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix

GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# iterative paramter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 50#10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number

DT = 0.2  # [s] time tick

# Vehicle parameters
WB = 110#2.5  # [m]

MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 100#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -100#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 1.0  # maximum accel [m/ss]

show_animation = True

def pi_2_pi(angle):
    while(angle > math.pi):
        angle = angle - 2.0 * math.pi

    while(angle < -math.pi):
        angle = angle + 2.0 * math.pi

    return angle

def check_goal(state, goal, tind, nind):

    # check goal
    dx = state.x - goal[0]
    dy = state.y - goal[1]
    d = math.sqrt(dx ** 2 + dy ** 2)

    isgoal = (d <= GOAL_DIS)

    if abs(tind - nind) >= 5:
        isgoal = False

    isstop = (abs(state.v) <= STOP_SPEED)

    if isgoal and isstop:
        return True

    return False

def smooth_yaw(yaw):

    for i in range(len(yaw) - 1):
        dyaw = yaw[i + 1] - yaw[i]

        while dyaw >= math.pi / 2.0:
            yaw[i + 1] -= math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

        while dyaw <= -math.pi / 2.0:
            yaw[i + 1] += math.pi * 2.0
            dyaw = yaw[i + 1] - yaw[i]

    return yaw


In [2]:
def iterative_linear_mpc_control(xref, Motion_Status, dref, oa, od,x0):
    """
    MPC contorl with updating operational point iteraitvely
    """  
    
    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T
    state=Motion_Status.get_state()   
    for i in range(MAX_ITER):
        xbar = Motion_Status.get_trajectory(oa, od,T)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, state, dref,x0)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")
    
    return oa, od, ox, oy, oyaw, ov,xbar

def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)#
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)#
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
    #C = np.zeros(NX)
    return A, B, C

def linear_mpc_control(xref, xbar, state, dref,x0):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], 0)
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t]+ C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <=
                            MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == [state.x,state.y,state.v,state.yaw]]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.OSQP, warm_start=True)
    #prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = np.array(x.value[0, :]).flatten()
        oy = np.array(x.value[1, :]).flatten()
        ov = np.array(x.value[2, :]).flatten()
        oyaw = np.array(x.value[3, :]).flatten()
        oa = np.array(u.value[0, :]).flatten()
        odelta = np.array(u.value[1, :]).flatten()

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov



In [3]:
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=state
    def get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = curr_state.x + curr_state.v * math.cos(curr_state.yaw) * DT
        next_y = curr_state.y + curr_state.v * math.sin(curr_state.yaw) * DT
        next_yaw = curr_state.yaw + curr_state.v / WB * math.tan(delta) * DT
        next_v = curr_state.v + a * DT
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion.get_next_state(self,accel, delta, curr_state=_state)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw
            _state=next_state
        

        return xbar


In [4]:
def gen_points(cx,cy):
    dx=20#2
    dy=10#1
    pts=[[cx-dx,cy+dy],
    [cx+dx,cy+dy],
    [cx+dx,cy-dy],
    [cx-dx,cy-dy],
    [cx-dx,cy+dy]]
    pts=np.array(pts,dtype=np.float64)
    return pts
class move_points():
    def __init__(self,points,XX,YY,phi):
        self.points=points.copy()
        self.XX=XX
        self.YY=YY
        self.ox=0
        self.oy=0
        phi=(math.pi*phi)/180
        self.rot_mat=np.array([[math.cos(phi),-math.sin(phi)],[math.sin(phi),math.cos(phi)]])
    def move(self):
        self.points[:,0]+=self.XX
        self.points[:,1]+=self.YY
    def get_origin(self):
        ox=0
        oy=0
        oo=np.mean(self.points,axis=0)
        self.ox=oo[0]
        self.oy=oo[1]
        print(self.ox,self.oy)
        return self.ox,self.oy
    def shift_origin(self):
        self.points[:,0]-=np.float64(self.ox)
        self.points[:,1]-=np.float64(self.oy)
        return self.points
    def reverse_shift_origin(self):
        self.points[:,0]+=self.ox
        self.points[:,1]+=self.oy
        return self.points
    def rotate(self):
        for i in range(len(self.points)):
            self.points[i]=self.points[i].dot(self.rot_mat)
        return self.points
    def apply(self):
        self.move()
        self.get_origin()
        self.shift_origin()
        self.rotate()
        self.reverse_shift_origin()
        return self.points

In [5]:
%matplotlib tk
import gym
from matplotlib import pyplot as plt
import cv2
import numpy as np
from car_env import CarRacing
import time
import cubic_spline_planner
def rad_to_deg():
    pass
def accl_to_gas(accl,vel):
    dt=1.0
    SIZE = 0.02
    ENGINE_POWER= 100000000*SIZE*SIZE
    WHEEL_MOMENT_OF_INERTIA = 4000*SIZE*SIZE
    WHEEL_R  = 27 
    wheel_rad=WHEEL_R*SIZE
    mass=(2*WHEEL_MOMENT_OF_INERTIA)/wheel_rad
    #mass=.25
    power=4*mass*accl*(2*vel+accl*dt)/2#
    #power=4*mass*accl*(vel+accl/2)
    gas=power/ENGINE_POWER
    return gas

def env_step(env,action,show_plot=False):
    
    frame=np.zeros((100,200,3),dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    #action[0]=action[0]-np.deg2rad(90)
    action[0]*=-1
    nst,rw,ter,info=envc.step(action) # take a random action
    frame[0:96,0:96,:]=np.array(nst)
    
    pos=envc.car.hull.position
    vel=np.sqrt(envc.car.hull.linearVelocity[0]**2+envc.car.hull.linearVelocity[1]**2)
    ang=envc.car.hull.angle
    #ang*=-1
    ang=ang+np.deg2rad(90)#-ang
    if show_plot==True:
        plt.cla()
        img = cv2.putText(np.array(frame), 'vel %f'%vel, (100,10), font,.3, (255,255,0),1, cv2.LINE_AA)
        img = cv2.putText(np.array(img), 'ang %f'%ang, (100,20), font,.3, (255,0,255),1, cv2.LINE_AA)
        image = cv2.putText(np.array(img), 'x,y %f,%f'%(pos[0],pos[1]), (100,30), font,.2, (255,0,255),1, cv2.LINE_AA)
       
        plt.imshow(image)
        plt.show()
        plt.pause(0.0001)
        
    return [pos[0],pos[1],ang,vel],rw,ter,info
#envc.close()


envc=CarRacing()
def env_reset(envc):
    import pickle
    envc.reset()
    save=False
    if save==True:
        f=open('path-1','wb')
        pickle.dump(envc.track,f)
        f.close()
    f=open('path-1','rb')
    track=pickle.load(f)
    f.close()
    envc.track=track
    
    points=[[x,y] for a,b,x,y in envc.track]
    points=np.array(points)
    plt.plot(points[:,0],points[:,1])
    plt.show()
    return [env_step(envc,[0,0,0]),points]
    
#st,points=env_reset(envc)
'''
for i in range(100):
    st,rw,ter,info=env_step(envc,[0,1,0])
    time.sleep(0.1)
    #print(st)
    '''
#envc.close()



'\nfor i in range(100):\n    st,rw,ter,info=env_step(envc,[0,1,0])\n    time.sleep(0.1)\n    #print(st)\n    '

In [6]:
def calc_speed_profile(cx, cy, cyaw, target_speed):

    speed_profile = [target_speed] * len(cx)
    direction = 1.0  # forward

    # Set stop point
    for i in range(len(cx) - 1):
        dx = cx[i + 1] - cx[i]
        dy = cy[i + 1] - cy[i]

        move_direction = math.atan2(dy, dx)

        if dx != 0.0 and dy != 0.0:
            dangle = abs(pi_2_pi(move_direction - cyaw[i]))
            if dangle >= math.pi / 4.0:
                direction = -1.0
            else:
                direction = 1.0

        if direction != 1.0:
            speed_profile[i] = - target_speed
        else:
            speed_profile[i] = target_speed

    speed_profile[-1] = 0.0

    return speed_profile



In [7]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
envc=CarRacing()
[init_state,traj_points]=env_reset(envc)
ix,iy,iyaw,ivel=init_state[0]
tx,ty=traj_points[:50,0],traj_points[:50,1]

cx, cy, _, _, _ = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
#yaw0=np.rad2deg(90)
#cyaw=[yaw-yaw0 for yaw in cyaw]

dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]
sp = calc_speed_profile(cx, cy, cyaw, TARGET_SPEED)


Motion_Status = Motion(x=ix, y=iy, yaw=iyaw, v=ivel)
state=Motion_Status.get_state()
print(state)

Track generation: 1127..1413 -> 286-tiles track


In [8]:
# initial yaw compensation
if state.yaw - cyaw[0] >= math.pi:
        state.yaw -= math.pi * 2.0
elif state.yaw - cyaw[0] <= -math.pi:
        state.yaw += math.pi * 2.0
Motion_Status.set_state(state)
goal = [cx[-1], cy[-1]]


from collections import deque
class Data_Streamer():
    
    def __init__(self,qlen):
        self.point_stream=deque(maxlen=qlen)
        self.qlen=qlen
        self.x=None
        self.y=None
        self.v=None
        self.yaw=None
        self.index=0
        self.dind=0
        self.end=False
        
    def get_planned_traj(self,cx,cy,sp,cyaw):
        self.x=cx
        self.y=cy
        self.v=sp
        self.yaw=cyaw
        self.index=0
        self.dind=0
        self.end=False
        for _ in range(self.qlen):
            ind=self.index
            self.point_stream.append([self.x[ind],self.y[ind],self.v[ind],self.yaw[ind]])
            self.index+=1
       
    def get_trajectory(self):
        return self.point_stream.copy()
    
    def add_points(self,num):
        
        for _ in range(num):
            ind=self.index
            if ind<len(cx):
                self.point_stream.append([self.x[ind],self.y[ind],self.v[ind],self.yaw[ind]])
                self.index+=1
            else:
                self.end=True
                break
        return self.end
   
    def get_origin_shift(self,state):
        
        traj_points=self.get_trajectory()
        d = [(state.x-idx) ** 2 + (state.y-idy) ** 2 for (idx, idy, _, _) in traj_points]
        mind = min(d)
        self.dind = d.index(mind)
        return self.dind
    
    def shift_trajectory(self):
        
        if self.dind:
            self.end=self.add_points(dind)
            
        return self.end
            
        
    def get_ref_trajectory(self,state,_dl,_dt,t_step):
        
        traj_points=self.get_trajectory()
        xref = np.zeros((NX, t_step + 1))
        dref = np.zeros((1, t_step + 1))
        ncourse = len(cx)

        xref[:, 0] = traj_points[0]
        dref[0, 0] = 0.0  # steer operational point should be 0

        travel = 0.0

        for i in range(t_step + 1):
            travel += abs(state.v) * _dt
            dind = int(round(travel / _dl))

            if (dind) < len(traj_points)-1:
                xref[:, i] = traj_points[i]
                dref[0, i] = 0.0
            else:
                xref[:, i] = traj_points[-1]
                dref[0, i] = 0.0

        return xref,dref


In [9]:
time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]





#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
_index=0
cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''

sensor_stream=Data_Streamer(15)
sensor_stream.get_planned_traj(cx,cy,sp,cyaw)
while MAX_TIME >= time:
        state=Motion_Status.get_state()
        x0 = [state.x, state.y, state.v, state.yaw]
        
        dind=  sensor_stream.get_origin_shift(state)
        end =  sensor_stream.shift_trajectory()
        if end==True:
            break
        xref, dref = sensor_stream.get_ref_trajectory(state,dl,DT,T)
        _index+=dind
        
        oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(xref, Motion_Status, dref, oa, odelta,x0)
        
        state_str=str(state.x)+','+str(state.y)+','+str(state.v)+','+str(np.rad2deg(state.yaw))+'\n'
        xref_str=str(xref[0][0])+','+str(xref[1][0])+','+str(xref[2][0])+','+str(np.rad2deg(xref[3][0]))+','+str(_index)+'\n'
        xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        act_str=str(accl)+','+str(gas)+','+str(np.rad2deg(steer))+'\n'
    
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        if odelta is not None:
            di, ai = odelta[0], oa[0]
        accl=ai
        gas=accl_to_gas(accl,state.v)
        steer=di
        
        #act_str=str(steer)+'\n'
        nst,rw,ter,info=env_step(envc,[steer,gas,0])
        next_state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        #next_state = Motion_Status.get_next_state(ai, di)
        Motion_Status.set_state(next_state)
        
        time = time + DT

        x.append(state.x)
        y.append(state.y)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        
        
        if show_animation:  # pragma: no cover
            plt.cla()
            plt.text(50,50,'state : '+state_str,fontsize=12)
            plt.text(50,40,'xref : '+xref_str,fontsize=12)
            plt.text(50,30,'xbar : '+xbar_str,fontsize=12)
            plt.text(50,20,'act : '+act_str,fontsize=12)
            
            if ox is not None:
                plt.plot(ox, oy, "xr", label="MPC")
            plt.plot(cx, cy, "-r", label="course")
            plt.plot(x, y, "ob", label="trajectory")
            plt.plot(xref[0, :], xref[1, :], "xk", label="xref")
            plt.plot(cx[_index], cy[_index], "xg", label="target")
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            points=gen_points(state.x,state.y)
            ang_yaw=-(state.yaw*180.0)/math.pi
            rp=move_points(points,0,0,ang_yaw)
            r_points=rp.apply()
            points=np.array(points,dtype=np.float64)
            r_points=np.array(r_points,dtype=np.float64)
            plt.plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            plt.title("Time[s]:" + str(round(time, 2))
                      + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
                      + '\nyaw'+str(state.yaw)
                      + '\nxref_yaw'+str(xref[3][0])
                      + '\nsteer'+str(steer))
            plt.pause(0.0001)

if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(cx, cy, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


213.67010498046875 -49.9587287902832
213.67010498046875 -49.95858383178711
213.67015075683594 -49.95832061767578
213.67019653320312 -49.95793533325195
213.67027282714844 -49.95742416381836
213.6703643798828 -49.956787109375
213.67047119140625 -49.956024169921875
213.67059326171875 -49.95513153076172
213.67074584960938 -49.9541130065918
213.67091369628906 -49.952964782714844
213.6710968017578 -49.951683044433594
213.67129516601562 -49.95027160644531
213.67152404785156 -49.948726654052734
213.67176818847656 -49.947044372558594
213.6720428466797 -49.945228576660156
213.67233276367188 -49.943275451660156
213.6726531982422 -49.94118118286133
213.67298889160156 -49.93894958496094
213.67335510253906 -49.93657684326172
213.67373657226562 -49.93406295776367
213.67416381835938 -49.9314079284668
213.67459106445312 -49.92860794067383
213.675048828125 -49.9256591796875
213.675537109375 -49.92256546020508
213.67604064941406 -49.9193229675293
213.6765899658203 -49.915931701660156
213.67715454101562 -

215.3098602294922 -45.31112289428711
215.3234405517578 -45.25745391845703
215.33538818359375 -45.20314025878906
215.34580993652344 -45.148311614990234
215.3566131591797 -45.0931510925293
215.36810302734375 -45.037681579589844
215.3802490234375 -44.9819221496582
215.39306640625 -44.925865173339844
215.40650939941406 -44.8695182800293
215.42051696777344 -44.8128776550293
215.43508911132812 -44.755943298339844
215.4501495361328 -44.698707580566406
215.46563720703125 -44.64116668701172
215.4815216064453 -44.583309173583984
215.4977569580078 -44.52513122558594
215.51429748535156 -44.46662139892578
215.5311279296875 -44.40777587890625
215.54641723632812 -44.348121643066406
215.55995178222656 -44.287742614746094
215.57164001464844 -44.226715087890625
215.58163452148438 -44.16517639160156
215.5922393798828 -44.103302001953125
215.60386657714844 -44.041114807128906
215.61643981933594 -43.97865295410156
215.6298828125 -43.91591262817383
215.6441192626953 -43.85289764404297
215.65904235839844 -43

218.4440155029297 -23.47905921936035
218.460693359375 -23.328824996948242
218.472900390625 -23.177766799926758
218.48028564453125 -23.026145935058594
218.48248291015625 -22.874279022216797
218.4876708984375 -22.7218074798584
218.49659729003906 -22.568809509277344
218.50970458984375 -22.41555404663086
218.52725219726562 -22.262277603149414
218.549560546875 -22.109294891357422
218.57688903808594 -21.95697021484375
Iterative is max iter
218.6095733642578 -21.805715560913086
218.63961791992188 -21.653200149536133
218.66627502441406 -21.499292373657227
Iterative is max iter
218.68902587890625 -21.344146728515625
218.70755004882812 -21.18790626525879
218.7214813232422 -21.030799865722656
Iterative is max iter
218.73045349121094 -20.87309455871582
Iterative is max iter
218.7340850830078 -20.71511459350586
Iterative is max iter
218.73194885253906 -20.557231903076172
Iterative is max iter
218.7244873046875 -20.399709701538086
218.71934509277344 -20.241315841674805
218.71726989746094 -20.0819740

218.07737731933594 12.789122581481934
217.92315673828125 12.936315536499023
217.7647705078125 13.0795316696167
217.6023406982422 13.21863842010498
217.43589782714844 13.353504180908203
217.26556396484375 13.484002113342285
217.0914306640625 13.610007286071777
216.91360473632812 13.731393814086914
216.73219299316406 13.848040580749512
Iterative is max iter
216.54730224609375 13.959830284118652
216.3590545654297 14.066646575927734
216.16964721679688 14.174455642700195
215.97955322265625 14.284165382385254
215.7893524169922 14.396369934082031
215.59945678710938 14.511604309082031
215.4104461669922 14.63035774230957
215.22291564941406 14.753107070922852
215.0375518798828 14.8803071975708
214.85508728027344 15.012397766113281
214.6763458251953 15.149785995483398
214.50218200683594 15.292841911315918
214.32891845703125 15.437405586242676
214.1520538330078 15.578838348388672
213.97116088867188 15.716222763061523
213.78604125976562 15.848886489868164
213.5966796875 15.97613525390625
213.403060

171.0937042236328 39.607242584228516
170.8245086669922 39.68330764770508
170.5555419921875 39.763851165771484
170.28746032714844 39.849876403808594
170.0210418701172 39.942386627197266
169.75714111328125 40.042388916015625
169.49676513671875 40.15087890625
169.24105834960938 40.26879119873047
168.99127197265625 40.396976470947266
168.7488555908203 40.53617477416992
168.51535034179688 40.68695831298828
168.28663635253906 40.84426498413086
168.06260681152344 41.008056640625
167.84361267089844 41.17830276489258
167.6298828125 41.35517501831055
167.42172241210938 41.538665771484375
167.21298217773438 41.723793029785156
167.0026397705078 41.90953826904297
166.79006958007812 42.094940185546875
166.57467651367188 42.27915954589844
166.35592651367188 42.461185455322266
166.13331604003906 42.63994216918945
165.90640258789062 42.814247131347656
165.67478942871094 42.98282241821289
165.4381561279297 43.14430236816406
165.1962890625 43.29722213745117
164.94908142089844 43.44004440307617
164.696670

In [10]:
_cx

NameError: name '_cx' is not defined

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=1.0)
i_yaw=cyaw[0]
cyaw=[yaw-i_yaw for yaw in cyaw]
cyaw

In [ ]:
#%matplotlib inline
plt.plot(cx,cy)
plt.show()

In [ ]:
plt.plot(range(len(cyaw)),cyaw)
plt.show()

In [ ]:
dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
yaw0=np.deg2rad(90)
cyaw=[yaw-yaw0 for yaw in cyaw]

In [ ]:
import numpy as np
np.rad2deg(2.64)

In [ ]:
cyaw

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 1
n = 1
np.random.seed(1)
A = 10#np.random.randn(m, n)
b = 5#np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A*x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

In [ ]:
print(x.value)

In [ ]:
x=np.array(range(0,100))
x=x/100

In [ ]:
y=10*x-5
y=np.square(y)
plt.plot(x,y)
plt.show()